In [46]:
from os import listdir
import os
from scipy import misc, stats, ndimage
import matplotlib.pyplot as plt
import numpy as np
from skimage.draw import line_aa
from tqdm import tqdm

def closest_nodes(node, nodes):
    nodes = np.asarray(nodes)
    dist_2 = np.sqrt(np.sum((nodes - node)**2, axis=1))
    result = nodes[np.argwhere(dist_2 <= threshold)]
    if len(result) == 1:
        return None
    else:
        return result

def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        
def precision(predicted, groundtruth):
    hits     = len(np.argwhere((predicted == groundtruth and predicted == 255)))
    returned = len(np.argwhere(predicted == 255))
    return hits / returned

def recall(predicted, groundtruth):
    hits     = len(np.argwhere((predicted == groundtruth and predicted == 255)))
    correct  = len(np.argwhere(groundtruth == 255))
    

In [52]:
samples_locations = './Output_samples/'
for threshold in (20,):
    print('starting threshold: {}'.format(threshold))
    for file in tqdm(listdir(samples_locations)):
        if file[0]=='.': continue
        img = misc.imread(samples_locations+file)
        eroded = ndimage.binary_erosion(img / 255).astype(img.dtype)
        thin = img - eroded*255
        pairs = []

        new_img = np.zeros(thin.shape)
        road_pxs = np.argwhere(thin == 1)

        for pixel_road in road_pxs:
            closest_pxs = closest_nodes(pixel_road, road_pxs)
            if closest_pxs != None:
                for pixel in closest_pxs:
                    pairs += [[pixel_road, pixel[0]]]
        for x, y in pairs:
            rr, cc, val = line_aa(x[0], x[1], y[0], y[1])
            new_img[rr, cc] = val * 255
        new_img = new_img + img
        new_img[new_img!=0] = 255
        create_dir('../results_smooth/{}'.format(threshold))
        misc.imsave('../results_smooth/{}/{}'.format(threshold, file), new_img)

  0%|          | 0/50 [00:00<?, ?it/s]

starting threshold: 20


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:16: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
100%|██████████| 50/50 [00:27<00:00,  1.72it/s]
